In [2]:
from ibapi.client import EClient
from ibapi.wrapper import EWrapper
from ibapi.contract import Contract
import threading
import time

In [3]:
tickers = ["AMZN","META","INTC"]

In [ ]:
class TradeApp(EWrapper, EClient):
    def __init__(self):
        EClient.__init__(self, self)
        self.lastPrice = {}
        
    def tickPrice(self, reqId, tickType, price, attrib):
        super().tickPrice(reqId, tickType, price, attrib)
        #print("TickPrice. TickerId:", reqId, "tickType:", tickType, "Price:", price)
        if tickType==4:
            self.lastPrice[reqId] = price

In [8]:
def usTechStk(symbol,sec_type="STK",currency="USD",exchange="ISLAND"):
    contract = Contract()
    contract.symbol = symbol
    contract.secType = sec_type
    contract.currency = currency
    contract.exchange = exchange
    return contract 

def streamSnapshotData(tickers):
    """stream tick leve data"""
    for ticker in tickers:
        app.reqMktData(reqId=tickers.index(ticker), 
                       contract=usTechStk(ticker),
                       genericTickList="",
                       snapshot=False,
                       regulatorySnapshot=False,
                       mktDataOptions=[])
        time.sleep(1)
    
def last_price(ticker):
    return app.lastPrice[tickers.index(ticker)]
    
def connection():
    app.run()

In [9]:
app = TradeApp()
app.connect(host='127.0.0.1', port=7497, clientId=23) #port 4002 for ib gateway paper trading/7497 for TWS paper trading

ConThread = threading.Thread(target=connection)
ConThread.start()

streamThread = threading.Thread(target=streamSnapshotData, args=(tickers,))
streamThread.start()
time.sleep(3) #some lag added to ensure that streaming has started


print("Last Price of AMZN:", last_price("AMZN"))

ERROR -1 2104 Market data farm connection is OK:usfarm.nj
ERROR -1 2104 Market data farm connection is OK:usfuture
ERROR -1 2104 Market data farm connection is OK:usopt.nj
ERROR -1 2104 Market data farm connection is OK:cashfarm
ERROR -1 2104 Market data farm connection is OK:usfarm
ERROR -1 2106 HMDS data farm connection is OK:ushmds
ERROR -1 2158 Sec-def data farm connection is OK:secdefil


Last Price of AMZN: 237.3


In [10]:
app.disconnect()